In [ ]:
!pip install pyspark
import pyspark
from pyspark import SparkConf, SparkContext 
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import avg
from pyspark.sql.functions import mean
from pyspark.sql.types import FloatType
from pyspark.sql.functions import col
from pyspark.sql.functions import udf
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
spark = SparkSession.builder.appName("Vancouver House Value prediction").getOrCreate()

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Formatted_Zillow.csv to Formatted_Zillow (6).csv


In [ ]:
from pyspark.sql.types import DoubleType
df = spark.read.csv("Formatted_Zillow.csv", header=True, inferSchema=True)
df = df.withColumn('Price', col("Price").cast(DoubleType()))
df.printSchema()

In [ ]:
df.show(5)

+-------------+---------+--------------------+--------------------+---------------+------+-------+-----+---------+-----------+--------------------+
|Type_Property|    Price|             Address|      Address_Street|Address_Zipcode|N_Beds|N_Baths| Area|      Lat|       Long|         Broker_Name|
+-------------+---------+--------------------+--------------------+---------------+------+-------+-----+---------+-----------+--------------------+
|        Condo| 429000.0|4990 McGeer St #1...| 4990 McGeer St #114|         V5R6C1|     1|      1|  527| 49.23952|-123.032814|  RE/MAX City Realty|
|        House|   5.98E7|4838 Belmont Ave,...|    4838 Belmont Ave|         V6T1A9|     5|      8|12410| 49.27489| -123.22121|    Macdonald Realty|
|        Condo| 558000.0|1216 W 11th Ave #...|1216 W 11th Ave #206|         V6H1K5|     1|      1|  865|49.261246| -123.13179|Sotheby's Interna...|
|        Condo| 638000.0|489 Interurban Wa...|489 Interurban Wa...|         V5X0C7|     1|      1|  492|      0.

In [ ]:
df.count()

800

In [ ]:
df.select(*[mean(c) for c in df.columns]).show()

+------------------+-------------+------------+-------------------+--------------------+------------------+------------------+----------+------------------+-------------------+----------------+
|avg(Type_Property)|   avg(Price)|avg(Address)|avg(Address_Street)|avg(Address_Zipcode)|       avg(N_Beds)|      avg(N_Baths)| avg(Area)|          avg(Lat)|          avg(Long)|avg(Broker_Name)|
+------------------+-------------+------------+-------------------+--------------------+------------------+------------------+----------+------------------+-------------------+----------------+
|              null|3164698.75375|        null|               null|                null|3.3404255319148937|3.0938673341677094|2196.09625|42.052280633750016|-105.11074844875006|            null|
+------------------+-------------+------------+-------------------+--------------------+------------------+------------------+----------+------------------+-------------------+----------------+



In [ ]:
df.groupby('Type_Property').agg({col: 'avg' for col in df.columns[0:17]}).show()

+-----------------+------------------+------------------+----------------+------------+------------------+------------------+-----------------+------------------+-------------------+-------------------+--------------------+
|    Type_Property|         avg(Area)|       avg(N_Beds)|avg(Broker_Name)|avg(Address)|      avg(N_Baths)|          avg(Lat)|       avg(Price)|avg(Type_Property)|          avg(Long)|avg(Address_Street)|avg(Address_Zipcode)|
+-----------------+------------------+------------------+----------------+------------+------------------+------------------+-----------------+------------------+-------------------+-------------------+--------------------+
|        Townhouse|           1460.36|              2.66|            null|        null|              2.62| 41.37400926999999|       1718102.61|              null|-103.41481089000003|               null|                null|
|       Lot / Land|           10880.0|              null|            null|        null|              nul

In [ ]:
train, test = df.randomSplit([0.7, 0.3])
train, test

(DataFrame[Type_Property: string, Price: double, Address: string, Address_Street: string, Address_Zipcode: string, N_Beds: int, N_Baths: int, Area: int, Lat: double, Long: double, Broker_Name: string],
 DataFrame[Type_Property: string, Price: double, Address: string, Address_Street: string, Address_Zipcode: string, N_Beds: int, N_Baths: int, Area: int, Lat: double, Long: double, Broker_Name: string])

In [ ]:
numerical_features_lst = train.columns
numerical_features_lst.remove('Price')
numerical_features_lst.remove('Address')
numerical_features_lst.remove('Address_Zipcode')
numerical_features_lst.remove('Address_Street')
numerical_features_lst.remove('Broker_Name')
numerical_features_lst.remove('Type_Property')
numerical_features_lst

['N_Beds', 'N_Baths', 'Area', 'Lat', 'Long']

In [ ]:
from pyspark.ml.feature import Imputer

imputer = Imputer(inputCols=numerical_features_lst,
                            outputCols=numerical_features_lst)
imputer = imputer.fit(train)
train = imputer.transform(train)
test = imputer.transform(test)
train.show(7)

+-------------+--------+--------------------+--------------------+---------------+------+-------+----+---------+-----------+--------------------+
|Type_Property|   Price|             Address|      Address_Street|Address_Zipcode|N_Beds|N_Baths|Area|      Lat|       Long|         Broker_Name|
+-------------+--------+--------------------+--------------------+---------------+------+-------+----+---------+-----------+--------------------+
|        Condo|319000.0|1250 Bur0by St #6...| 1250 Bur0by St #606|         V6E1P6|     1|      1| 560|      0.0|        0.0|Sutton Group-West...|
|        Condo|349000.0|1250 Burnaby St #...|1250 Burnaby St #607|         V6E1P6|     1|      1| 570| 49.28147|-123.135605|Sutton Group-West...|
|        Condo|378000.0|1850 Comox St #10...| 1850 Comox St #1008|         V6G1R3|     1|      1| 546|      0.0|        0.0|Royal Pacific Tri...|
|        Condo|399000.0|1219 Harwood St #...|1219 Harwood St #603|         V6E1S5|     1|      1| 540|      0.0|        0.0|

In [ ]:
from pyspark.ml.feature import VectorAssembler

numerical_vector_assembler = VectorAssembler(inputCols=numerical_features_lst,
                                             outputCol='numerical_feature_vector')

train = numerical_vector_assembler.transform(train)
test = numerical_vector_assembler.transform(test)
train.show(2)

+-------------+--------+--------------------+--------------------+---------------+------+-------+----+--------+-----------+--------------------+------------------------+
|Type_Property|   Price|             Address|      Address_Street|Address_Zipcode|N_Beds|N_Baths|Area|     Lat|       Long|         Broker_Name|numerical_feature_vector|
+-------------+--------+--------------------+--------------------+---------------+------+-------+----+--------+-----------+--------------------+------------------------+
|        Condo|319000.0|1250 Bur0by St #6...| 1250 Bur0by St #606|         V6E1P6|     1|      1| 560|     0.0|        0.0|Sutton Group-West...|    [1.0,1.0,560.0,0....|
|        Condo|349000.0|1250 Burnaby St #...|1250 Burnaby St #607|         V6E1P6|     1|      1| 570|49.28147|-123.135605|Sutton Group-West...|    [1.0,1.0,570.0,49...|
+-------------+--------+--------------------+--------------------+---------------+------+-------+----+--------+-----------+--------------------+------

In [ ]:
train.select('numerical_feature_vector').take(2)

[Row(numerical_feature_vector=DenseVector([1.0, 1.0, 560.0, 0.0, 0.0])),
 Row(numerical_feature_vector=DenseVector([1.0, 1.0, 570.0, 49.2815, -123.1356]))]

In [ ]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol = 'numerical_feature_vector',
                        outputCol= 'scaled_numerical_feature_vector',
                        withStd= True, withMean=True)

scaler = scaler.fit(train)

train = scaler.transform(train)
test = scaler.transform(test)

train.show(3)

+-------------+--------+--------------------+--------------------+---------------+------+-------+----+--------+-----------+--------------------+------------------------+-------------------------------+
|Type_Property|   Price|             Address|      Address_Street|Address_Zipcode|N_Beds|N_Baths|Area|     Lat|       Long|         Broker_Name|numerical_feature_vector|scaled_numerical_feature_vector|
+-------------+--------+--------------------+--------------------+---------------+------+-------+----+--------+-----------+--------------------+------------------------+-------------------------------+
|        Condo|319000.0|1250 Bur0by St #6...| 1250 Bur0by St #606|         V6E1P6|     1|      1| 560|     0.0|        0.0|Sutton Group-West...|    [1.0,1.0,560.0,0....|           [-1.2422016680011...|
|        Condo|349000.0|1250 Burnaby St #...|1250 Burnaby St #607|         V6E1P6|     1|      1| 570|49.28147|-123.135605|Sutton Group-West...|    [1.0,1.0,570.0,49...|           [-1.24220166

In [ ]:
train.select('scaled_numerical_feature_vector').take(3)

[Row(scaled_numerical_feature_vector=DenseVector([-1.2422, -1.0608, -0.8575, -2.5265, 2.5265])),
 Row(scaled_numerical_feature_vector=DenseVector([-1.2422, -1.0608, -0.8524, 0.3966, -0.3955])),
 Row(scaled_numerical_feature_vector=DenseVector([-1.2422, -1.0608, -0.8647, -2.5265, 2.5265]))]

In [ ]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol ='Type_Property', outputCol = 'Type_Property_index' )

indexer = indexer.fit(train)
train = indexer.transform(train)
test = indexer.transform(test)

train.show(3)

+-------------+--------+--------------------+--------------------+---------------+------+-------+----+--------+-----------+--------------------+------------------------+-------------------------------+-------------------+
|Type_Property|   Price|             Address|      Address_Street|Address_Zipcode|N_Beds|N_Baths|Area|     Lat|       Long|         Broker_Name|numerical_feature_vector|scaled_numerical_feature_vector|Type_Property_index|
+-------------+--------+--------------------+--------------------+---------------+------+-------+----+--------+-----------+--------------------+------------------------+-------------------------------+-------------------+
|        Condo|319000.0|1250 Bur0by St #6...| 1250 Bur0by St #606|         V6E1P6|     1|      1| 560|     0.0|        0.0|Sutton Group-West...|    [1.0,1.0,560.0,0....|           [-1.2422016680011...|                0.0|
|        Condo|349000.0|1250 Burnaby St #...|1250 Burnaby St #607|         V6E1P6|     1|      1| 570|49.28147|-

In [ ]:
set(train.select('Type_Property_index').collect())

{Row(Type_Property_index=0.0),
 Row(Type_Property_index=1.0),
 Row(Type_Property_index=2.0),
 Row(Type_Property_index=3.0),
 Row(Type_Property_index=4.0)}

In [ ]:
from pyspark.ml.feature import OneHotEncoder

one_hot_encoder = OneHotEncoder(inputCol='Type_Property_index', outputCol = 'Type_Property_One_Hot' )

one_hot_encoder = one_hot_encoder.fit(train)

train = one_hot_encoder.transform(train)
test = one_hot_encoder.transform(test)

train.show(3)

+-------------+--------+--------------------+--------------------+---------------+------+-------+----+--------+-----------+--------------------+------------------------+-------------------------------+-------------------+---------------------+
|Type_Property|   Price|             Address|      Address_Street|Address_Zipcode|N_Beds|N_Baths|Area|     Lat|       Long|         Broker_Name|numerical_feature_vector|scaled_numerical_feature_vector|Type_Property_index|Type_Property_One_Hot|
+-------------+--------+--------------------+--------------------+---------------+------+-------+----+--------+-----------+--------------------+------------------------+-------------------------------+-------------------+---------------------+
|        Condo|319000.0|1250 Bur0by St #6...| 1250 Bur0by St #606|         V6E1P6|     1|      1| 560|     0.0|        0.0|Sutton Group-West...|    [1.0,1.0,560.0,0....|           [-1.2422016680011...|                0.0|        (4,[0],[1.0])|
|        Condo|349000.0|

In [ ]:
assembler = VectorAssembler(inputCols=['scaled_numerical_feature_vector', 'Type_Property_One_Hot'],
                            outputCol='final_feature_vector')

train = assembler.transform(train)
test = assembler.transform(test)
train.show(2)

+-------------+--------+--------------------+--------------------+---------------+------+-------+----+--------+-----------+--------------------+------------------------+-------------------------------+-------------------+---------------------+--------------------+
|Type_Property|   Price|             Address|      Address_Street|Address_Zipcode|N_Beds|N_Baths|Area|     Lat|       Long|         Broker_Name|numerical_feature_vector|scaled_numerical_feature_vector|Type_Property_index|Type_Property_One_Hot|final_feature_vector|
+-------------+--------+--------------------+--------------------+---------------+------+-------+----+--------+-----------+--------------------+------------------------+-------------------------------+-------------------+---------------------+--------------------+
|        Condo|319000.0|1250 Bur0by St #6...| 1250 Bur0by St #606|         V6E1P6|     1|      1| 560|     0.0|        0.0|Sutton Group-West...|    [1.0,1.0,560.0,0....|           [-1.2422016680011...|    

In [ ]:
train.select('final_feature_vector').take(2)

[Row(final_feature_vector=DenseVector([-1.2422, -1.0608, -0.8575, -2.5265, 2.5265, 1.0, 0.0, 0.0, 0.0])),
 Row(final_feature_vector=DenseVector([-1.2422, -1.0608, -0.8524, 0.3966, -0.3955, 1.0, 0.0, 0.0, 0.0]))]

In [ ]:
from pyspark.ml.regression import  LinearRegression

lr = LinearRegression(featuresCol = 'final_feature_vector', labelCol='Price')

lr

LinearRegression_4206fb647d68

In [ ]:
lr = lr.fit(train)
lr

LinearRegressionModel: uid=LinearRegression_4206fb647d68, numFeatures=9

In [ ]:
pred_train_df = lr.transform(train).withColumnRenamed('prediction', 'Predicted_House_Price')
pred_train_df.show(5)

+-------------+--------+--------------------+--------------------+---------------+------+-------+----+---------+-----------+--------------------+------------------------+-------------------------------+-------------------+---------------------+--------------------+---------------------+
|Type_Property|   Price|             Address|      Address_Street|Address_Zipcode|N_Beds|N_Baths|Area|      Lat|       Long|         Broker_Name|numerical_feature_vector|scaled_numerical_feature_vector|Type_Property_index|Type_Property_One_Hot|final_feature_vector|Predicted_House_Price|
+-------------+--------+--------------------+--------------------+---------------+------+-------+----+---------+-----------+--------------------+------------------------+-------------------------------+-------------------+---------------------+--------------------+---------------------+
|        Condo|319000.0|1250 Bur0by St #6...| 1250 Bur0by St #606|         V6E1P6|     1|      1| 560|      0.0|        0.0|Sutton Group

In [ ]:
pred_test_df = lr.transform(test).withColumnRenamed('prediction', 'Predicted_House_Price')
pred_test_df.show(5)

+-------------+--------+--------------------+--------------------+---------------+------+-------+----+---------+-----------+--------------------+------------------------+-------------------------------+-------------------+---------------------+--------------------+---------------------+
|Type_Property|   Price|             Address|      Address_Street|Address_Zipcode|N_Beds|N_Baths|Area|      Lat|       Long|         Broker_Name|numerical_feature_vector|scaled_numerical_feature_vector|Type_Property_index|Type_Property_One_Hot|final_feature_vector|Predicted_House_Price|
+-------------+--------+--------------------+--------------------+---------------+------+-------+----+---------+-----------+--------------------+------------------------+-------------------------------+-------------------+---------------------+--------------------+---------------------+
|        Condo|339000.0|1251 Cardero St #...|1251 Cardero St #806|         V6G2H9|     1|      1| 436|      0.0|        0.0|Coldwell Ban

In [ ]:
pred_test_pd_df = pred_test_df.toPandas()
pred_test_pd_df.head(2)

,Type_Property,Price,Address,Address_Street,Address_Zipcode,N_Beds,N_Baths,Area,Lat,Long,Broker_Name,numerical_feature_vector,scaled_numerical_feature_vector,Type_Property_index,Type_Property_One_Hot,final_feature_vector,Predicted_House_Price
0,Condo,339000.0,"1251 Cardero St #806, Vancouver, BC V6G 2H9",1251 Cardero St #806,V6G2H9,1,1,436,0.0000,0.00000,Coldwell Banker Prestige Realty,"[1.0, 1.0, 436.0, 0.0, 0.0]","[-1.2422016680011334, -1.0607991189354384, -0....",0.0,"(1.0, 0.0, 0.0, 0.0)","[-1.2422016680011334, -1.0607991189354384, -0....",437382.538987
1,Condo,349000.0,"1219 Harwood St #604, Vancouver, BC V6E 1S5",1219 Harwood St #604,V6E1S5,1,1,547,49.2808,-123.13561,Century 21 In Town Realty,"[1.0, 1.0, 547.0, 49.2808, -123.13561]","[-1.2422016680011334, -1.0607991189354384, -0....",0.0,"(1.0, 0.0, 0.0, 0.0)","[-1.2422016680011334, -1.0607991189354384, -0....",793096.231714


In [ ]:
predictions_and_actuals = pred_test_df[['Predicted_House_Price','Price']]
predictions_and_actuals_rdd = predictions_and_actuals.rdd
predictions_and_actuals_rdd.take(3)

[Row(Predicted_House_Price=437382.53898741305, Price=339000.0),
 Row(Predicted_House_Price=793096.2317139432, Price=349000.0),
 Row(Predicted_House_Price=1030928.0085978992, Price=380000.0)]

In [ ]:
predictions_and_actuals = predictions_and_actuals_rdd.map(tuple)
predictions_and_actuals_rdd.take(2)

[Row(Predicted_House_Price=437382.53898741305, Price=339000.0),
 Row(Predicted_House_Price=793096.2317139432, Price=349000.0)]

In [ ]:
from pyspark.mllib.evaluation import RegressionMetrics
metrics = RegressionMetrics(predictions_and_actuals_rdd)

s = '''
Mean Squeard Error:      {0}
Root Mean Squeard Error: {1}
Mean Absolute Error:     {2}
R**2:                    {3}
'''.format(metrics.meanSquaredError,
           metrics.rootMeanSquaredError,
           metrics.meanAbsoluteError,
           metrics.r2)
print(s)

/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(



Mean Squeard Error:      7540412179856.261
Root Mean Squeard Error: 2745981.096048598
Mean Absolute Error:     1078876.1753957355
R**2:                    0.7232783849229282

